In [9]:
import json
import numpy as np
import pandas as pd

In [3]:
file = "big5_training_data.csv"
data = pd.read_csv(file, header=0)
data.head()

,message_id,message,ope_z,con_z,ext_z,agr_z,neu_z
0,tweet_143508,""";;tengo mi penchi fierron up in the air & its...",-0.300409,0.459249,0.294330,0.452765,-0.894041
1,tweet_425070,perhaps he loved everyone but me. nbut that ma...,0.896169,0.493197,-0.414594,0.337368,0.393312
2,tweet_502775,"if you have 3 coffee cups in your car, you hav...",0.213276,0.176036,0.328285,0.274713,-0.211072
3,tweet_600640,i'm too sleepy to start work in an hour,-0.711772,1.284832,-1.013234,0.611918,0.545335
4,blog_79108,i worked for a recruiter a little bit after i ...,1.837614,0.042504,-0.460709,0.061335,1.808086


In [5]:
data.shape

(846304, 7)

In [8]:
col = "ope_z"
p33, p66 = data[col].quantile([0.3333, 0.6666])
print(f"33%: {p33} and 66%: {p66}")

33%: -0.40305181453621464 and 66%: 0.3032634845677586


In [14]:
def label_by_percentile(value, p33, p66):
    if value <= p33:
        return 0
    elif value <= p66:
        return 1
    else:
        return 2

def dataset_filter(df):
    z_columns = [col for col in df.columns if col.endswith('_z')]
    
    for col in z_columns:
        p33, p66 = df[col].quantile([0.3333, 0.6666])
        print(f"Column: {col}, the first threshold: {p33}, the second threshold: {p66}")
        df[f'{col}_label'] = df[col].apply(lambda x: label_by_percentile(x, p33, p66))
    
    return df

filtered_df = dataset_filter(data)

Column: ope_z, the first threshold: -0.40305181453621464, the second threshold: 0.3032634845677586
Column: con_z, the first threshold: -0.317019038156446, the second threshold: 0.39680313301426884
Column: ext_z, the first threshold: -0.3480345697823196, the second threshold: 0.3458212935858872
Column: agr_z, the first threshold: -0.24902418364823875, the second threshold: 0.42130983790841997
Column: neu_z, the first threshold: -0.41538778569093865, the second threshold: 0.21921024917447135


In [18]:
print(filtered_df.isnull().sum())

message_id     0
message        0
ope_z          0
con_z          0
ext_z          0
agr_z          0
neu_z          0
ope_z_label    0
con_z_label    0
ext_z_label    0
agr_z_label    0
neu_z_label    0
dtype: int64


In [20]:
filter_data = filtered_df.drop(columns=['message_id', "ope_z", "con_z", "ext_z", "agr_z", "neu_z"])
filter_data

,message,ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
0,""";;tengo mi penchi fierron up in the air & its...",1,2,1,2,0
1,perhaps he loved everyone but me. nbut that ma...,2,2,0,1,2
2,"if you have 3 coffee cups in your car, you hav...",1,1,1,1,1
3,i'm too sleepy to start work in an hour,0,2,0,2,2
4,i worked for a recruiter a little bit after i ...,2,1,0,1,2
...,...,...,...,...,...,...
846299,hey what do u think of 'unwell' by matchbox 20...,1,1,2,2,0
846300,my life is dedicated to school right now ready...,0,2,2,2,0
846301,bush and cheney continue to assert that there ...,2,1,1,0,0
846302,sometimes my in-laws do things that are totall...,2,1,2,1,1


In [21]:
import pandas as pd

def check_label_distribution(df, label_columns):
    total_rows = len(df)
    expected_count = total_rows / 3

    for col in label_columns:
        label_counts = df[col].value_counts()
        print(f"Label distribution for {col}:")
        print(label_counts)

        if len(label_counts) != 3:
            print(f"Warning: {col} does not have exactly 3 labels.")

        for label, count in label_counts.items():
            ratio = count / total_rows
            if abs(ratio - 1/3) > 0.01:
                print(f"Warning: Label {label} in {col} has a ratio of {ratio:.2f}, which is not close to 1/3.")

        print("\n")


label_columns = [col for col in filter_data.columns if col.endswith('_label')]
check_label_distribution(filter_data, label_columns)

Label distribution for ope_z_label:
ope_z_label
2    282158
1    282073
0    282073
Name: count, dtype: int64


Label distribution for con_z_label:
con_z_label
2    282158
1    282073
0    282073
Name: count, dtype: int64


Label distribution for ext_z_label:
ext_z_label
2    282158
1    282073
0    282073
Name: count, dtype: int64


Label distribution for agr_z_label:
agr_z_label
2    282158
1    282073
0    282073
Name: count, dtype: int64


Label distribution for neu_z_label:
neu_z_label
2    282158
0    282073
1    282073
Name: count, dtype: int64




In [22]:
filter_data.to_csv("filtered_big5_data_3_label.csv", index=True)

In [23]:
filter_data # filtered_big5_data_3_label

,message,ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
0,""";;tengo mi penchi fierron up in the air & its...",1,2,1,2,0
1,perhaps he loved everyone but me. nbut that ma...,2,2,0,1,2
2,"if you have 3 coffee cups in your car, you hav...",1,1,1,1,1
3,i'm too sleepy to start work in an hour,0,2,0,2,2
4,i worked for a recruiter a little bit after i ...,2,1,0,1,2
...,...,...,...,...,...,...
846299,hey what do u think of 'unwell' by matchbox 20...,1,1,2,2,0
846300,my life is dedicated to school right now ready...,0,2,2,2,0
846301,bush and cheney continue to assert that there ...,2,1,1,0,0
846302,sometimes my in-laws do things that are totall...,2,1,2,1,1
